In [1]:
import pandas as pd
import pickle

In [56]:
def load_patient_data(file_path):
    if not file_path.endswith(".csv"):
        raise ValueError("File must be CSV Format")
    df = pd.read_csv(file_path)
    if len(df) != 1:
        raise ValueError("File must contain exactly ONE patient record.")
    return df

def get_medical_interpretation(prediction, disease_prob, patient_features):
    age = patient_features['age']
    chol = patient_features['chol']
    bp = patient_features['trestbps']
    interpretation = {
        'risk_level': '',
        'recommendations': [],
        'risk_factors': []
    }
    if prediction == 1:
        if disease_prob > 0.8:
            interpretation['risk_level'] = 'HIGH RISK'
            interpretation['recommendations'] = [
                "Immediate cardiology consultation",
                "Emergency ECG and tests",
                "Consider hospital admission"
            ]
        elif disease_prob > 0.6:
            interpretation['risk_level'] = 'MODERATE-HIGH RISK'
            interpretation['recommendations'] = [
                "Urgent cardiology follow-up",
                "ECG and cardiac workup",
                "Lifestyle modifications"
            ]
        else:
            interpretation['risk_level'] = 'MODERATE RISK'
            interpretation['recommendations'] = [
                "Schedule cardiology appointment",
                "Basic cardiac screening",
                "Regular monitoring"
            ]
    else:
        if disease_prob < 0.2:
            interpretation['risk_level'] = 'LOW RISK'
            interpretation['recommendations'] = [
                "Routine annual check-up",
                "Maintain healthy lifestyle",
                "Continue preventive care"
            ]
        elif disease_prob < 0.4:
            interpretation['risk_level'] = 'LOW-MODERATE RISK'
            interpretation['recommendations'] = [
                "Regular health monitoring",
                "Focus on risk factor reduction",
                "Annual cardiac screening"
            ]
        else:
            interpretation['risk_level'] = 'BORDERLINE RISK'
            interpretation['recommendations'] = [
                "Enhanced monitoring",
                "Address risk factors",
                "Repeat assessment in 3-6 months"
            ]
    if age > 45:
        interpretation['risk_factors'].append(f"Age {age} (>45 years)")
    if chol > 200:
        interpretation['risk_factors'].append(f"Cholesterol {chol} (>200 mg/dL)")
    if bp > 140:
        interpretation['risk_factors'].append(f"Blood Pressure {bp} (>140 mmHg)")
    return interpretation

def generate_report(patient_info, prediction, prob, interpretation):
    rec_text = "\n - ".join([""] + interpretation['recommendations'])
    risk_text = "\n - ".join([""] + interpretation['risk_factors']) if interpretation['risk_factors'] else "None"
    report_text = (
        "========== HEART DISEASE PREDICTION REPORT ==========\n\n"
        "PATIENT INFORMATION:\n"
        f"{patient_info.to_string(index=False, justify='center')}\n\n"
        "PREDICTION:\n"
        f"HEART DISEASE: {'HEART DISEASE DETECTED' if prediction == 1 else 'NO HEART DISEASE DETECTED'}\n"
        f"DISEASE PROBABILITY: {prob:.4f}\n"
        f"RISK LEVEL: {interpretation['risk_level']}\n\n"
        f"RISK FACTORS:\n{risk_text}\n\n"
        f"RECOMMENDATIONS:\n{rec_text}\n\n"
)
    return report_text

def run_prediction(file_path):
    df = load_patient_data(file_path)
    FEATURE_ORDER = [
        "age", "sex", "cp", "trestbps", "chol",
        "fbs", "restecg", "thalach", "exang",
        "oldpeak", "slope", "ca", "thal"
    ]
    df_ordered = df[FEATURE_ORDER]
    model = pickle.load(open("Heart_Disease_Predictor.pkl", "rb"))
    y_pred = model.predict(df_ordered)[0]
    y_prob = model.predict_proba(df_ordered)[:, 1][0]
    interpretation = get_medical_interpretation(y_pred,y_prob,df.iloc[0].to_dict())
    report = generate_report(df, y_pred, y_prob, interpretation)
    with open("Heart_Disease_Report.txt", "w") as file:
        file.write(report)
    print("Report generated successfully: Heart_Disease_Report.txt")
    
run_prediction("new_patient.csv")

Report generated successfully: Heart_Disease_Report.txt
